# Lab 3: Advanced Tools & Resources

**Objective:** Master advanced MCP patterns including async tools with progress reporting, resources with URI patterns, and programmatic MCP client access.

**Duration:** ~45 minutes

**What You'll Learn:**
- How to create async tools with progress reporting
- How to expose data through MCP resources
- How to connect to MCP servers programmatically as a client

## Part 1: Async Tools with Progress Reporting

For long-running operations, async tools let you report progress to the user.

In [ ]:
# Server with async tools and progress reporting

async_server_code = '''
import asyncio
from mcp.server.fastmcp import FastMCP, Context

mcp = FastMCP("Async Tools Server")

@mcp.tool()
async def process_data(items: int, ctx: Context) -> str:
    """Process a number of items with progress updates.
    
    Args:
        items: Number of items to process
        ctx: MCP context for progress reporting
    
    Returns:
        Summary of processed items
    """
    processed = 0
    
    for i in range(items):
        # Simulate work
        await asyncio.sleep(0.5)
        processed += 1
        
        # Report progress
        await ctx.report_progress(
            progress=processed,
            total=items,
            message=f"Processed {processed}/{items} items"
        )
    
    return f"Successfully processed {processed} items"

@mcp.tool()
async def fetch_data(url: str, ctx: Context) -> str:
    """Simulate fetching data from a URL with progress.
    
    Args:
        url: The URL to fetch (simulated)
        ctx: MCP context for progress reporting
    
    Returns:
        The fetched data content
    """
    await ctx.report_progress(0, 100, "Connecting...")
    await asyncio.sleep(1)
    
    await ctx.report_progress(30, 100, "Downloading...")
    await asyncio.sleep(1)
    
    await ctx.report_progress(80, 100, "Parsing...")
    await asyncio.sleep(0.5)
    
    await ctx.report_progress(100, 100, "Complete!")
    
    return f"Data fetched from {url}: [Simulated content]"

if __name__ == "__main__":
    mcp.run()
'''

with open("async_server.py", "w") as f:
    f.write(async_server_code)

print("Async server saved to async_server.py")
print("\nKey patterns demonstrated:")
print("- async def for async tools")
print("- Context parameter for progress reporting")
print("- await ctx.report_progress() for updates")

## Challenge 1: Test Async Tools

**TODO:**
1. Run `mcp dev async_server.py`
2. Test `process_data` with items=5
3. Observe progress updates in MCP Inspector
4. Test `fetch_data` with any URL

In [ ]:
# TODO: Document your async tool testing

async_test_results = {
    "process_data_test": {
        "items_input": None,
        "progress_updates_seen": None,  # True/False
        "final_result": "",
        "observations": ""
    },
    "fetch_data_test": {
        "url_input": "",
        "progress_stages_seen": [],  # ["Connecting", "Downloading", etc.]
        "final_result": ""
    }
}

print("Fill in your async tool test results")

## Part 2: MCP Resources

Resources expose read-only data through URI patterns. Think of them like GET endpoints in a REST API.

In [ ]:
# Server with resources

resource_server_code = '''
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("Resource Server")

# Sample data store
DOCUMENTS = {
    "readme": "# Welcome\\nThis is the readme document.",
    "guide": "# User Guide\\nStep 1: Install\\nStep 2: Configure\\nStep 3: Run",
    "faq": "# FAQ\\nQ: How do I start?\\nA: Run the main script."
}

USERS = {
    "alice": {"name": "Alice", "role": "admin", "email": "alice@example.com"},
    "bob": {"name": "Bob", "role": "user", "email": "bob@example.com"}
}

# Resource with URI pattern
@mcp.resource("doc://{doc_id}")
def get_document(doc_id: str) -> str:
    """Get a document by its ID.
    
    Args:
        doc_id: The document identifier (readme, guide, faq)
    
    Returns:
        The document content
    """
    if doc_id not in DOCUMENTS:
        return f"Document \'{doc_id}\' not found. Available: {list(DOCUMENTS.keys())}"
    return DOCUMENTS[doc_id]

@mcp.resource("user://{username}")
def get_user(username: str) -> str:
    """Get user information by username.
    
    Args:
        username: The username to look up
    
    Returns:
        User information as formatted string
    """
    if username not in USERS:
        return f"User \'{username}\' not found. Available: {list(USERS.keys())}"
    user = USERS[username]
    return f"Name: {user[\'name\']}\\nRole: {user[\'role\']}\\nEmail: {user[\'email\']}"

# List resource for discovery
@mcp.resource("doc://list")
def list_documents() -> str:
    """List all available documents."""
    return "Available documents: " + ", ".join(DOCUMENTS.keys())

# Tool that uses resources
@mcp.tool()
def search_documents(query: str) -> str:
    """Search across all documents for a query.
    
    Args:
        query: The search term
    
    Returns:
        Documents containing the query
    """
    results = []
    for doc_id, content in DOCUMENTS.items():
        if query.lower() in content.lower():
            results.append(doc_id)
    
    if not results:
        return f"No documents found containing \'{query}\'"
    return f"Found \'{query}\' in: {\', \'.join(results)}"

if __name__ == "__main__":
    mcp.run()
'''

with open("resource_server.py", "w") as f:
    f.write(resource_server_code)

print("Resource server saved to resource_server.py")
print("\nResources defined:")
print("- doc://{doc_id} - Get document by ID")
print("- user://{username} - Get user info")
print("- doc://list - List all documents")
print("\nTools defined:")
print("- search_documents(query) - Search across documents")

## Challenge 2: Test Resources

**TODO:**
1. Run `mcp dev resource_server.py`
2. In MCP Inspector, find the Resources tab
3. Test accessing `doc://readme`
4. Test accessing `user://alice`
5. Try an invalid resource like `doc://invalid`

In [ ]:
# TODO: Document your resource testing

resource_test_results = {
    "doc_readme": {
        "uri": "doc://readme",
        "content_received": "",
        "success": None
    },
    "user_alice": {
        "uri": "user://alice",
        "content_received": "",
        "success": None
    },
    "invalid_resource": {
        "uri": "doc://invalid",
        "error_message": "",
        "handled_gracefully": None
    },
    "search_tool_test": {
        "query": "",
        "result": ""
    }
}

print("Fill in your resource test results")

## Part 3: MCP Client SDK - Programmatic Access

You can also connect to MCP servers programmatically from your Python code.

In [ ]:
# MCP Client example - connecting to an MCP server programmatically

client_code = '''
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def main():
    # Configure server parameters
    server_params = StdioServerParameters(
        command="python",
        args=["calculator_server.py"]  # The server we created in Lab 2
    )
    
    # Connect to the server
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()
            
            # List available tools
            tools_response = await session.list_tools()
            print("Available tools:")
            for tool in tools_response.tools:
                print(f"  - {tool.name}: {tool.description}")
            
            # Call a tool
            result = await session.call_tool(
                "add",
                arguments={"a": 10, "b": 5}
            )
            print(f"\\nResult of add(10, 5): {result.content[0].text}")
            
            # Call another tool
            result = await session.call_tool(
                "multiply",
                arguments={"a": 7, "b": 8}
            )
            print(f"Result of multiply(7, 8): {result.content[0].text}")

if __name__ == "__main__":
    asyncio.run(main())
'''

with open("mcp_client_example.py", "w") as f:
    f.write(client_code)

print("MCP client example saved to mcp_client_example.py")
print("\nThis demonstrates:")
print("- Connecting to an MCP server from Python")
print("- Listing available tools")
print("- Calling tools programmatically")

### Running the Client Example

To test the client:

1. Make sure `calculator_server.py` from Lab 2 is in the same directory
2. Run: `python mcp_client_example.py`

The client will:
1. Start the calculator server as a subprocess
2. Connect via stdio transport
3. List tools and call them

## Challenge 3: Build a Client for the Resource Server

**TODO:** Write MCP client code that:
1. Connects to `resource_server.py`
2. Lists available resources
3. Reads a document resource
4. Calls the search_documents tool

In [ ]:
# TODO: Complete this client code

resource_client_code = '''
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def main():
    server_params = StdioServerParameters(
        command="python",
        args=["resource_server.py"]
    )
    
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # TODO: List available resources
            # resources = await session.list_resources()
            # print("Available resources:")
            # for resource in resources.resources:
            #     print(f"  - {resource.uri}")
            
            # TODO: Read a specific resource
            # content = await session.read_resource("doc://readme")
            # print(f"\\nReadme content: {content.contents[0].text}")
            
            # TODO: Call the search tool
            # result = await session.call_tool(
            #     "search_documents",
            #     arguments={"query": "install"}
            # )
            # print(f"\\nSearch result: {result.content[0].text}")
            
            pass  # Remove this when you implement the TODOs

if __name__ == "__main__":
    asyncio.run(main())
'''

with open("resource_client.py", "w") as f:
    f.write(resource_client_code)

print("Resource client template saved to resource_client.py")
print("TODO: Uncomment and complete the implementation!")

## Part 4: Combining Everything

Let's create a server that combines async tools, resources, and proper error handling.

In [ ]:
# Advanced server combining all concepts

advanced_server_code = '''
import asyncio
import json
from mcp.server.fastmcp import FastMCP, Context

mcp = FastMCP("Advanced Knowledge Server")

# Data store
KNOWLEDGE_BASE = {
    "python": {
        "title": "Python Programming",
        "content": "Python is a high-level programming language known for its simplicity.",
        "tags": ["programming", "scripting", "data science"]
    },
    "mcp": {
        "title": "Model Context Protocol",
        "content": "MCP is a protocol for connecting AI applications to external tools and data.",
        "tags": ["ai", "protocol", "tools"]
    },
    "fastmcp": {
        "title": "FastMCP SDK",
        "content": "FastMCP is the Python SDK for building MCP servers with decorators.",
        "tags": ["mcp", "python", "sdk"]
    }
}

# Resources
@mcp.resource("kb://{topic}")
def get_knowledge(topic: str) -> str:
    """Get knowledge base entry by topic."""
    if topic not in KNOWLEDGE_BASE:
        return f"Topic not found. Available: {list(KNOWLEDGE_BASE.keys())}"
    entry = KNOWLEDGE_BASE[topic]
    return f"# {entry[\'title\']}\\n\\n{entry[\'content\']}\\n\\nTags: {\', \'.join(entry[\'tags\'])}"

@mcp.resource("kb://list")
def list_topics() -> str:
    """List all knowledge base topics."""
    return json.dumps(list(KNOWLEDGE_BASE.keys()))

# Async tool with progress
@mcp.tool()
async def deep_search(query: str, ctx: Context) -> str:
    """Perform a deep search across the knowledge base with progress updates.
    
    Args:
        query: Search term to find
        ctx: Context for progress reporting
    
    Returns:
        Search results with relevance scores
    """
    results = []
    topics = list(KNOWLEDGE_BASE.keys())
    
    for i, topic in enumerate(topics):
        await ctx.report_progress(i, len(topics), f"Searching {topic}...")
        await asyncio.sleep(0.5)  # Simulate processing
        
        entry = KNOWLEDGE_BASE[topic]
        score = 0
        
        # Simple relevance scoring
        if query.lower() in entry["title"].lower():
            score += 3
        if query.lower() in entry["content"].lower():
            score += 2
        if query.lower() in " ".join(entry["tags"]).lower():
            score += 1
        
        if score > 0:
            results.append({"topic": topic, "score": score})
    
    await ctx.report_progress(len(topics), len(topics), "Search complete!")
    
    if not results:
        return f"No results found for \'{query}\'"
    
    results.sort(key=lambda x: x["score"], reverse=True)
    return "\\n".join([f"- {r[\'topic\']} (score: {r[\'score\']})" for r in results])

# Sync tool with validation
@mcp.tool()
def add_knowledge(topic: str, title: str, content: str, tags: str) -> str:
    """Add a new entry to the knowledge base.
    
    Args:
        topic: Unique identifier for the topic (lowercase, no spaces)
        title: Display title for the entry
        content: The main content text
        tags: Comma-separated list of tags
    
    Returns:
        Confirmation message
    """
    # Validation
    if not topic or " " in topic:
        return "Error: Topic must be non-empty with no spaces"
    if topic in KNOWLEDGE_BASE:
        return f"Error: Topic \'{topic}\' already exists"
    if not title or not content:
        return "Error: Title and content are required"
    
    # Add entry
    KNOWLEDGE_BASE[topic] = {
        "title": title,
        "content": content,
        "tags": [t.strip() for t in tags.split(",")] if tags else []
    }
    
    return f"Successfully added \'{topic}\' to knowledge base"

if __name__ == "__main__":
    mcp.run()
'''

with open("advanced_server.py", "w") as f:
    f.write(advanced_server_code)

print("Advanced server saved to advanced_server.py")
print("\nFeatures:")
print("- Resources with URI patterns")
print("- Async tool with progress reporting")
print("- Input validation")
print("- Error handling")

## Challenge 4: Extend the Advanced Server

**TODO:** Add a new feature to the advanced server:

1. Add a `get_related_topics(topic: str)` tool that finds topics with overlapping tags
2. Add a resource `kb://{topic}/summary` that returns just the title and tag count

In [ ]:
# TODO: Write the code for the new features

# New tool: get_related_topics
get_related_tool = '''
@mcp.tool()
def get_related_topics(topic: str) -> str:
    """Find topics related to the given topic based on shared tags.
    
    Args:
        topic: The topic to find relations for
    
    Returns:
        List of related topics with shared tag count
    """
    # TODO: Implement this
    pass
'''

# New resource: summary
summary_resource = '''
@mcp.resource("kb://{topic}/summary")
def get_summary(topic: str) -> str:
    """Get a brief summary of a topic."""
    # TODO: Implement this
    pass
'''

print("TODO: Implement these features and add them to advanced_server.py")
print("\nget_related_topics tool:")
print(get_related_tool)
print("\nget_summary resource:")
print(summary_resource)

## Lab Summary

In this lab, you learned:

1. **Async Tools**: How to create long-running tools with progress reporting
2. **Resources**: How to expose data through URI-based resources
3. **MCP Client SDK**: How to connect to servers programmatically
4. **Combined Patterns**: Building servers with multiple advanced features

### Key Takeaways

- Use async tools for operations that take time
- Progress reporting improves user experience
- Resources are for read-only data access
- The MCP Client SDK enables programmatic integration
- Validation and error handling are essential

### Next Lab

In Lab 4, you'll build a complete Document Assistant MCP server!

## Part 5: Payments Industry Example - Transaction Analytics Server

Let's build a more realistic MCP server that demonstrates async tools and resources in a payments context. This server provides transaction analytics with progress reporting for batch operations.

In [ ]:
# Payments Industry Example: Transaction Analytics MCP Server
# Demonstrates async tools with progress, resources, and real-world patterns

transaction_analytics_server = '''
import asyncio
import json
from datetime import datetime, timedelta
from mcp.server.fastmcp import FastMCP, Context

mcp = FastMCP("Transaction Analytics Server")

# Simulated transaction database
TRANSACTIONS = {
    "TX-001": {"amount": 150.00, "merchant": "Coffee Shop", "category": "Food", "status": "completed", "date": "2024-01-15", "customer_id": "C001"},
    "TX-002": {"amount": 5000.00, "merchant": "Electronics Plus", "category": "Electronics", "status": "completed", "date": "2024-01-15", "customer_id": "C002"},
    "TX-003": {"amount": 75.50, "merchant": "Gas Station", "category": "Gas", "status": "completed", "date": "2024-01-16", "customer_id": "C001"},
    "TX-004": {"amount": 1200.00, "merchant": "Online Store", "category": "Shopping", "status": "pending", "date": "2024-01-16", "customer_id": "C003"},
    "TX-005": {"amount": 45.00, "merchant": "Restaurant", "category": "Food", "status": "completed", "date": "2024-01-17", "customer_id": "C001"},
    "TX-006": {"amount": 8500.00, "merchant": "Jewelry Store", "category": "Luxury", "status": "flagged", "date": "2024-01-17", "customer_id": "C002"},
    "TX-007": {"amount": 23.99, "merchant": "Streaming Service", "category": "Subscription", "status": "completed", "date": "2024-01-18", "customer_id": "C001"},
    "TX-008": {"amount": 350.00, "merchant": "Airline", "category": "Travel", "status": "completed", "date": "2024-01-18", "customer_id": "C003"},
}

# Customer profiles
CUSTOMERS = {
    "C001": {"name": "Alice Smith", "tier": "gold", "risk_score": 15, "account_age_days": 730},
    "C002": {"name": "Bob Johnson", "tier": "platinum", "risk_score": 45, "account_age_days": 1825},
    "C003": {"name": "Carol White", "tier": "standard", "risk_score": 25, "account_age_days": 180},
}

# ============ RESOURCES ============

@mcp.resource("txn://{transaction_id}")
def get_transaction(transaction_id: str) -> str:
    """Get details for a specific transaction."""
    if transaction_id not in TRANSACTIONS:
        return f"Transaction {transaction_id} not found"
    txn = TRANSACTIONS[transaction_id]
    return json.dumps({
        "id": transaction_id,
        **txn
    }, indent=2)

@mcp.resource("customer://{customer_id}")
def get_customer(customer_id: str) -> str:
    """Get customer profile information."""
    if customer_id not in CUSTOMERS:
        return f"Customer {customer_id} not found"
    return json.dumps({
        "id": customer_id,
        **CUSTOMERS[customer_id]
    }, indent=2)

@mcp.resource("txn://list")
def list_transactions() -> str:
    """List all transaction IDs."""
    return json.dumps(list(TRANSACTIONS.keys()))

@mcp.resource("analytics://summary")
def get_summary() -> str:
    """Get overall transaction summary statistics."""
    total = sum(t["amount"] for t in TRANSACTIONS.values())
    count = len(TRANSACTIONS)
    by_status = {}
    for t in TRANSACTIONS.values():
        status = t["status"]
        by_status[status] = by_status.get(status, 0) + 1
    
    return json.dumps({
        "total_amount": total,
        "transaction_count": count,
        "average_amount": total / count if count > 0 else 0,
        "by_status": by_status
    }, indent=2)

# ============ ASYNC TOOLS WITH PROGRESS ============

@mcp.tool()
async def analyze_customer_transactions(customer_id: str, ctx: Context) -> str:
    """Analyze all transactions for a specific customer with detailed breakdown.
    
    Args:
        customer_id: The customer ID to analyze (e.g., C001)
        ctx: MCP context for progress reporting
    
    Returns:
        Comprehensive analysis of customer's transaction history
    """
    if customer_id not in CUSTOMERS:
        return f"Customer {customer_id} not found"
    
    customer = CUSTOMERS[customer_id]
    customer_txns = {tid: t for tid, t in TRANSACTIONS.items() if t["customer_id"] == customer_id}
    
    if not customer_txns:
        return f"No transactions found for customer {customer_id}"
    
    await ctx.report_progress(0, 4, "Gathering transaction data...")
    await asyncio.sleep(0.5)
    
    # Calculate totals
    total_amount = sum(t["amount"] for t in customer_txns.values())
    await ctx.report_progress(1, 4, "Calculating totals...")
    await asyncio.sleep(0.5)
    
    # Analyze by category
    by_category = {}
    for t in customer_txns.values():
        cat = t["category"]
        by_category[cat] = by_category.get(cat, 0) + t["amount"]
    await ctx.report_progress(2, 4, "Analyzing spending patterns...")
    await asyncio.sleep(0.5)
    
    # Risk assessment
    high_value_count = sum(1 for t in customer_txns.values() if t["amount"] > 1000)
    flagged_count = sum(1 for t in customer_txns.values() if t["status"] == "flagged")
    await ctx.report_progress(3, 4, "Assessing risk indicators...")
    await asyncio.sleep(0.5)
    
    await ctx.report_progress(4, 4, "Analysis complete!")
    
    return f"""
========================================
CUSTOMER TRANSACTION ANALYSIS
========================================
Customer: {customer['name']} ({customer_id})
Tier: {customer['tier'].upper()}
Account Age: {customer['account_age_days']} days
Base Risk Score: {customer['risk_score']}

TRANSACTION SUMMARY
-------------------
Total Transactions: {len(customer_txns)}
Total Amount: ${total_amount:,.2f}
Average Transaction: ${total_amount/len(customer_txns):,.2f}

SPENDING BY CATEGORY
--------------------
{chr(10).join(f"  {cat}: ${amt:,.2f}" for cat, amt in sorted(by_category.items(), key=lambda x: -x[1]))}

RISK INDICATORS
---------------
High-Value Transactions (>$1000): {high_value_count}
Flagged Transactions: {flagged_count}
Risk Assessment: {'ELEVATED' if flagged_count > 0 or high_value_count > 2 else 'NORMAL'}
========================================
"""

@mcp.tool()
async def batch_fraud_scan(ctx: Context) -> str:
    """Scan all transactions for potential fraud indicators with progress updates.
    
    Args:
        ctx: MCP context for progress reporting
    
    Returns:
        Fraud scan report with flagged transactions
    """
    suspicious = []
    txn_list = list(TRANSACTIONS.items())
    
    for i, (tid, txn) in enumerate(txn_list):
        await ctx.report_progress(i, len(txn_list), f"Scanning {tid}...")
        await asyncio.sleep(0.3)  # Simulate analysis time
        
        risk_factors = []
        
        # Check for high value
        if txn["amount"] > 5000:
            risk_factors.append(f"High value: ${txn['amount']:,.2f}")
        
        # Check for luxury category
        if txn["category"] == "Luxury":
            risk_factors.append("Luxury category purchase")
        
        # Check if already flagged
        if txn["status"] == "flagged":
            risk_factors.append("Previously flagged")
        
        # Check customer risk score
        customer = CUSTOMERS.get(txn["customer_id"], {})
        if customer.get("risk_score", 0) > 40:
            risk_factors.append(f"High-risk customer (score: {customer['risk_score']})")
        
        if risk_factors:
            suspicious.append({
                "transaction_id": tid,
                "amount": txn["amount"],
                "merchant": txn["merchant"],
                "risk_factors": risk_factors
            })
    
    await ctx.report_progress(len(txn_list), len(txn_list), "Scan complete!")
    
    if not suspicious:
        return "✅ FRAUD SCAN COMPLETE: No suspicious transactions detected."
    
    report = "🚨 FRAUD SCAN REPORT\\n" + "=" * 40 + "\\n\\n"
    report += f"Suspicious Transactions Found: {len(suspicious)}\\n\\n"
    
    for item in suspicious:
        report += f"Transaction: {item['transaction_id']}\\n"
        report += f"  Amount: ${item['amount']:,.2f}\\n"
        report += f"  Merchant: {item['merchant']}\\n"
        report += f"  Risk Factors:\\n"
        for rf in item['risk_factors']:
            report += f"    ⚠️ {rf}\\n"
        report += "\\n"
    
    return report

@mcp.tool()
async def generate_reconciliation_report(date: str, ctx: Context) -> str:
    """Generate a reconciliation report for a specific date.
    
    Args:
        date: The date to generate report for (YYYY-MM-DD format)
        ctx: MCP context for progress reporting
    
    Returns:
        Detailed reconciliation report
    """
    await ctx.report_progress(0, 5, "Filtering transactions...")
    await asyncio.sleep(0.4)
    
    day_txns = {tid: t for tid, t in TRANSACTIONS.items() if t["date"] == date}
    
    if not day_txns:
        return f"No transactions found for {date}"
    
    await ctx.report_progress(1, 5, "Calculating totals...")
    await asyncio.sleep(0.4)
    
    completed = [t for t in day_txns.values() if t["status"] == "completed"]
    pending = [t for t in day_txns.values() if t["status"] == "pending"]
    flagged = [t for t in day_txns.values() if t["status"] == "flagged"]
    
    await ctx.report_progress(2, 5, "Grouping by merchant...")
    await asyncio.sleep(0.4)
    
    by_merchant = {}
    for t in day_txns.values():
        m = t["merchant"]
        if m not in by_merchant:
            by_merchant[m] = {"count": 0, "total": 0}
        by_merchant[m]["count"] += 1
        by_merchant[m]["total"] += t["amount"]
    
    await ctx.report_progress(3, 5, "Calculating fees...")
    await asyncio.sleep(0.4)
    
    total_completed = sum(t["amount"] for t in completed)
    processing_fee = total_completed * 0.029  # 2.9% processing fee
    
    await ctx.report_progress(4, 5, "Generating report...")
    await asyncio.sleep(0.4)
    
    await ctx.report_progress(5, 5, "Report complete!")
    
    return f"""
================================================
DAILY RECONCILIATION REPORT - {date}
================================================

TRANSACTION SUMMARY
-------------------
Total Transactions: {len(day_txns)}
  ✅ Completed: {len(completed)}
  ⏳ Pending: {len(pending)}
  🚨 Flagged: {len(flagged)}

FINANCIAL SUMMARY
-----------------
Completed Amount: ${total_completed:,.2f}
Pending Amount: ${sum(t['amount'] for t in pending):,.2f}
Flagged Amount: ${sum(t['amount'] for t in flagged):,.2f}

Processing Fee (2.9%): ${processing_fee:,.2f}
Net Settlement: ${total_completed - processing_fee:,.2f}

BY MERCHANT
-----------
{chr(10).join(f"  {m}: {d['count']} txns, ${d['total']:,.2f}" for m, d in sorted(by_merchant.items(), key=lambda x: -x[1]['total']))}

================================================
Report generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
================================================
"""

if __name__ == "__main__":
    mcp.run()
'''

# Save the payments analytics server
with open("transaction_analytics_server.py", "w") as f:
    f.write(transaction_analytics_server)

print("Transaction Analytics Server saved to transaction_analytics_server.py")
print("\n" + "="*60)
print("PAYMENTS INDUSTRY MCP SERVER")
print("="*60)
print("\nRESOURCES (Read-only data access):")
print("  - txn://{transaction_id}  - Get transaction details")
print("  - customer://{customer_id} - Get customer profile")
print("  - txn://list              - List all transaction IDs")
print("  - analytics://summary     - Overall statistics")
print("\nASYNC TOOLS (With progress reporting):")
print("  - analyze_customer_transactions() - Customer spending analysis")
print("  - batch_fraud_scan()              - Scan all transactions for fraud")
print("  - generate_reconciliation_report() - Daily settlement report")
print("\nRun with: mcp dev transaction_analytics_server.py")